# **PIBS** - Permutational Invariance Block Solver
PIBS provides a highly efficient numerical method to calculate exact dynamics for models of many identical spins interacting with a common bosonic mode that have a certain weak $\text{U}(1)$ symmetry, whereby terms in the Liouvillian create or destroy at most one excitation. For such models, in the presence of non-unitary processes describing _only_ loss (or gain), the dynamics of blocks of states of fixed excitation number can be solved sequentially, starting from the highest (or lowest) block. This simplification is in addition to working in the permutation symmetric basis for the spins, which already provides a combinatoric reduction of the size of the problem to be solved.

In this notebook we explain the key functions and their signatures in PIBS, illustrating the use of the method applied to study dynamical superradiance from a single-mode cavity.

## Model
The most general model implemented in PIBS is the Tavis-Cummings model with local loss and dephasing processes

$$ H = \omega_c a^\dagger a +\sum_{n=1}^N\left[\frac{\omega_0}{2}\sigma_n^z + g\left(a\sigma_n^+ + a^\dagger\sigma_n^-\right)\right],$$
$$ \partial_t\rho = -\text{i}[H,\rho] + \kappa \mathcal L[a] + \sum_{n=1}^{N}\left(\gamma\mathcal L[\sigma_n^-]+\gamma_\phi\mathcal L[\sigma_n^z]\right), \mathcal{L}[X] = X\rho X^\dagger -(X^\dagger X\rho+\rho X^\dagger X)/2$$

where $a, a^\dagger$ are bosonic annihilation and creation operators for the cavity mode at frequency $\omega_c$, and $\sigma_n^z, \sigma_n^\pm$ are Pauli matrices for the $n$th two-level system with level splitting $\omega_0$ satisfying $[\sigma_n^z,\sigma_m^\pm] = \pm2\sigma^\pm$; $\kappa$ is the cavity loss, $\gamma$ the individual two-level system loss and $\gamma_\phi$ the individual dephasing rate.

## Setup
There are three key classes in `setup.py`, which define a PIBS problem:
- `Indices`, which enumerates the chosen spin basis elements in the permutation
  invariant problem
- `BlockL`, which provides all possible block components of the Liouvillian for
  models with weak symmetry
- `Models`, which defines the block Liouvillian form for specific models, e.g. Tavis-Cummings model

This separation allows parts of the computation to be performed independently, stored on disk, and used in multiple calculations. Two problems with the same number of spins will use the same `Indices` and `BlockL` objects, for example.

The final 'model' class `Models` allows all parts to be conveniently performed in one go, however: to setup the problem for the Tavis-Cummings model described above,one simply invokes

In [ ]:
import numpy as np
import os
from pibs import Models, Indices, Rho
from pibs import TimeEvolve
from pibs import basis, create,destroy, qeye, tensor, sigmam, sigmap, sigmaz


nspins =10  # number 2LS
w0 = 1.0    # energy difference 2LS
wc = 0.65   # cavity mode frequency 
Omega = 0.4
g = Omega / np.sqrt(nspins) # light-matter coupling
kappa = 1e-02     # photon loss
gamma = 1e-03     # exciton loss
gamma_phi =3e-02  # dephasing

# set up states in supercompressed form 
index_path = "data/indices/" 
indices = Indices(nspins, index_path = index_path, debug=False,save=True)  

# set up Liouvillian
liouv_path = "data/liouvillians/"
blockTC = Models(wc, w0,g, kappa, gamma_phi,gamma, indices, parallel=0,progress=False,debug=False, save=True,liouv_path=liouv_path )
blockTC.setup_L_Tavis_Cummings(progress = False)

The calculation will proceed starting from the creation of a suitable `Indices` object. The code will automatically check `DATA?SETUP_DIR` for `Indices` and `BlockL` constructed in previous calculations.  Similarly, it will automatically write to these directories during the calculation [Provide option to disable this / change save directories].

Here `nspins` is the number of identical spins coupled to the photon mode. The latter is represented in a finite basis of `nphot=nspins+1` levels, which is sufficient to fully characterise the dynamics for this type of problem. Currently, only the above Tavis-Cummings is implemented in PIBS, with `ldim_s=2` (two-level systems).

When calling `Models`, a Liouvillian basis will be calculated and stored for later use. Using the basis and the system parameters provided, the method `setup_L_Tavis_Cummings` calculates the Liouvillian in block form.

### Density matrix
The class `Rho` contains all the information concerning the density matrix of the system. The initial state as well as the reduced density matrix representation are calculated when initializing an object as follows:

In [ ]:
# initial condition with zero photons and all spins up.
rho = Rho(basis(nspins+1,0), basis(2,0), indices) 

The function `basis(N,n)` creates a Fock density matrix for an $N$-level Hilbert space, with an excitation in level $n$. So, `basis(nspins+1,0)` creates a density matrix for the photon space, with zero photons, and `basis(2,0)` creates a density matrix for a single spin in the excited state (note the convention that for the spin state $0=\uparrow$, $1=\downarrow$) 

## Time Evolution
The functionality of the time evolution is contained in the class `TimeEvolve` in `propagate.py`.

### Basic concept
Due to the weak U(1) symmetry, the master equation can be written in the following block form

$$\partial_t \rho^{(\nu,\nu')} = L_0^{(\nu,\nu')}\rho^{(\nu,\nu')} + L_1^{(\nu,\nu')}\rho^{(\nu+1,\nu'+1)},$$

where the "block" $\rho^{(\nu,\nu')}$ is a vector containing all density matrix elements (up to permutation symmetry) with $\nu$ excitations in the left state and $\nu'$ excitations in the right state, and $L_0^{(\nu,\nu')}, L_1^{(\nu,\nu')}$ are matrices. For example, the element $\langle 0,\downarrow\uparrow|\rho|1,\downarrow\uparrow\rangle$ is in the block with $\nu=1, \nu'=2$. The above evolution equations then simply states that only blocks with constant $\nu-\nu'$ couple in the dynamics. The matrices $L_0$ describe coherent dynamics and dephasing processes, while $L_1$ describe losses.

If one is interested in the average number of photons in the cavity or the population of the spins, then one only needs to solve the diagonal block $\nu=\nu'$, and the evolution equation is given by
    $$\partial_t\rho^{(\nu)} = L_0^{(\nu)} \rho^{(\nu)} + L_1^{(\nu)} \rho^{(\nu+1)},$$
where the $\nu'$ index is suppressed. This is the equation implemented in PIBS.

There exists a block with the highest excitation number, which in the superfluoresence case is $\nu_\text{max}=\text{nspins}$. This block has no coupling $L_1$ to a block with higher excitation numbers. Therefore, the time evolution starts by computing the states $\rho^{\nu_\text{max}}$, and then the states for smaller $\nu$ can be solved iteratively.

### Operators
For the time evolution, we can define operators whose expectaion values we want to compute for each time step. In our examples, we calculate the expectation values of the number of photons  and of the number of excited spins 

$$\begin{align} \langle a^\dagger a\rangle &= \text{Tr}[\rho_1a^\dagger a \otimes 1_2]\\
   \langle\sigma^+\sigma^-\rangle &= \text{Tr}[\rho_1 1_\text{phot}\otimes\sigma^+\sigma^-]
   \end{align}
   $$
where $\rho_1$ is the reduced density matrix for one spin and the photon space.

In the code, we define the operators as follows:

In [ ]:
n = tensor(create(nspins+1)*destroy(nspins+1), qeye(2)) # photon number operator
p = tensor(qeye(nspins+1), sigmap()*sigmam())   # spin excitation
ops=[n,p]

Note that if one wants to calculate the expectation value of e.g. $\langle \sigma^+\rangle$, one has to solve the master equation for an off-diagonal block with $\nu-\nu'=1$ instead of $\nu-\nu'=0$. This is not implemented currently.

### Time-evolution methods
There are two different time-evolution methods available in the `TimeEvolve` class:
- `time_evolve_block_interp`: Serial time evolution, that first computes the state $\rho^{\nu_\text{max}}$ for all times, then feeding the results into the computation of the state $\rho^{\nu_\text{max-1}}$ and so on.
- `time_evolve_chunk_parallel`: Parallel time evolution in chunks, using the `multiprocessing.Pool()` function.

In the code, we create an instance of the class `TimeEvolve`, which has access to the two time-evolution methods.

#### Serial time evolution

In [ ]:
tmax = 200 # end of time evolution
dt = 0.2 # time step
evolve = TimeEvolve(rho, blockTC, tmax, dt, atol=1e-8, rtol=1e-8)

In [ ]:
evolve.time_evolve_block_interp(ops) 
n_serial = evolve.result.expect[0].real / nspins
p_serial = evolve.result.expect[1].real
t_serial = evolve.result.t

#### Parallel time evolution

In [ ]:
# parallel time evolution in chunks, using multiprocessing. DOES NOT WORK ON WINDOWS
chunksize=200
evolve.time_evolve_chunk_parallel(ops, chunksize=chunksize) 
n_parallel_mp = evolve.result.expect[0].real / nspins
p_parallel_mp = evolve.result.expect[1].real
t_parallel_mp = evolve.result.t

### Plot results

In [ ]:
import matplotlib.pyplot as plt
fig,ax = plt.subplots(1,2, figsize=(8,4), dpi=150)
fig.suptitle(r'$N={nspins},\ \Delta={delta},\ g\sqrt{{N}}={Omega},\ \kappa={kappa},\ \gamma={gamma},\ \gamma_\phi={gamma_phi}$'.format(nspins=nspins,delta=wc-w0, Omega=Omega,kappa=kappa,gamma=gamma,gamma_phi=gamma_phi))

ax[0].plot(t_serial, n_serial, label='serial')
ax[0].plot(t_parallel_mp, n_parallel_mp,'--', label=f'parallel mp ({chunksize})')
ax[1].plot(t_serial, p_serial, label='serial')
ax[1].plot(t_parallel_mp, p_parallel_mp,'--', label=f'parallel mp ({chunksize})')

ax[0].set_ylabel(r'$\langle n\rangle / N$')
ax[0].set_xlabel(r'$t$')
ax[1].set_ylabel(r'$\langle \sigma^+\sigma^-\rangle$')
ax[1].set_xlabel(r'$t$')

ax[0].legend()
ax[1].legend()
plt.tight_layout()
plt.show()